In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('../../data/csl/CSL_tytl_PI.csv', index_col=0)
#corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSiteCorr.csv', index_col=0, header=None, delimiter='\t')
#corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSite_abCont_u50.csv', index_col=0, header=None, delimiter='\t')
corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSite_abCont_ALL.csv', index_col=0, header=None, delimiter='\t')
corr_vars = list(corr_df.index.values)

In [3]:
%%time
from mwb_bootstrap import bootstrap_stat
from mwb_bootstrap import bootstrap_stat_all
from sklearn.ensemble import GradientBoostingClassifier

X = df.drop('trans_loss', axis=1, inplace=False)
X = X[corr_vars]
y = df['trans_loss'].values
    
clf = GradientBoostingClassifier(n_estimators=70, criterion="friedman_mse",max_depth=11, 
                          min_samples_leaf=50, min_samples_split=900,max_leaf_nodes=None,
                          max_features=12,subsample=0.9, learning_rate=0.1)
#                          max_features=12,subsample=0.9, learning_rate=0.1,random_state=7)

#stats_df, feats_df, X_train, y_train, X_test, y_test, y_pred = bootstrap_stat_all(X, y, clf, test_size=0.30, 
stats_df, feats_df, X_train, y_train, X_test, y_test, y_pred = bootstrap_stat_all(X, y, clf, test_size=0.20, 
                                                               sample_weights=True, nsamples=10, under=False)
stats_df.loc['mean'] = stats_df.mean()
stats_df.loc['mean',:]

CPU times: user 2min 39s, sys: 3.5 s, total: 2min 43s
Wall time: 2min 43s


recall     0.874315
prec       0.137553
MCC        0.264076
PR_AUC     0.228090
roc_auc    0.838146
Name: mean, dtype: float64

In [5]:
stats_df

,recall,prec,MCC,PR_AUC,roc_auc
0,0.877216,0.136866,0.263553,0.226673,0.839196
1,0.880761,0.137891,0.266173,0.229356,0.836556
2,0.886239,0.138728,0.268917,0.234877,0.841690
3,0.871737,0.138116,0.264449,0.216330,0.836136
4,0.872382,0.137022,0.262695,0.224684,0.836889
5,0.863680,0.136352,0.259498,0.228142,0.832322
6,0.866258,0.136565,0.260469,0.219824,0.837482
7,0.874315,0.138242,0.265272,0.235186,0.839324
8,0.867870,0.137076,0.261737,0.228849,0.838540
9,0.882694,0.138677,0.267993,0.236979,0.843322


In [6]:
feats_df['mean'] = feats_df.mean(axis=1)
feats_df['median'] = feats_df.median(axis=1)
feats_df['std'] = feats_df.std(axis=1)
sort_feat = feats_df.sort_values(by='mean', ascending=False)
sort_feat[['mean','median','std']].head(10)

,mean,median,std
Delmode,0.174265,0.174265,0.027252
Inoxy_incrdose,0.043990,0.040045,0.012982
Intratocolytix,0.038696,0.038696,0.008633
CS_FTP,0.029794,0.029794,0.006948
DMControl,0.027182,0.027182,0.007157
Insurance,0.027057,0.029031,0.005564
TrialLabor,0.026307,0.026043,0.010659
Hxanemia,0.025138,0.025138,0.003507
Lac_Min,0.023982,0.023982,0.009379
Education,0.022383,0.021994,0.005569


In [8]:
y_train_pred = clf.predict(X_train)
train_stats =  calc_stats(y_train, y_train_pred, X_train, clf)
train_stats 

recall     0.908852
prec       0.142240
MCC        0.280337
PR_AUC     0.301485
roc_auc    0.876292
dtype: float64

In [10]:
from mwb_bootstrap import bootstrap_stat_samp
import random
from sklearn.model_selection import train_test_split

samp_rate = []
mask = random.sample(range(len(X)), 30000)
X_samp = X.iloc[mask]
y_samp = y[mask]
y_count = np.bincount(y_samp)
#print(y_count)
samp_rate.append(y_count[1]/len(y_samp))
#print(f'samp_rate = {samp_rate}')


In [11]:
yt_counts = np.bincount(y_train)
print(yt_counts[1]/len(y_train))

0.055790552358058076


In [13]:
from mwb_bootstrap import calc_stats
samp_ypred = clf.predict(X_samp)
stats_s = calc_stats(y_samp, samp_ypred, X_samp, clf)
stats_s

recall     0.892980
prec       0.144871
MCC        0.277348
PR_AUC     0.288290
roc_auc    0.866357
dtype: float64

# Note: loop below hides variables stats_df & feats_df

In [16]:
site_stats = pd.DataFrame()
site_feats = pd.DataFrame()

from mwb_bootstrap import calc_stats
from sklearn.metrics import confusion_matrix
#pd.options.display.float_format = '{:,.4f}'.format
np.set_printoptions(precision=3)
# Loop through Sites
for site in [41, 44, 48, 49, 51]:
    site_df = df[df['Sitenum'] == site]
    X_site = site_df.drop('trans_loss', axis=1, inplace=False)
    X_site = X_site[corr_vars]
    y_site = site_df['trans_loss'].values
    ys_counts = np.bincount(y_site)
    print(f'y_pos[{site}]: {(ys_counts[1]/len(y_site))*100}%')
                 
    site_ypred = clf.predict(X_site)
    stats_s = calc_stats(y_site, site_ypred, X_site, clf, conf_flag=True)
    site_stats[site] = stats_s
#    print(f'\nSite{site} confusion matrix:\n{confusion_matrix(y_site, site_ypred)}')
#    print(f'\nSite{site} confusion matrix:\n{confusion_matrix(y_site, site_ypred, normalize="all")}')
    #tn, fp, fn, tp = confusion_matrix(y_site, site_ypred)
    #print(f'tn,fp,fn,tp[{site}]: {tn,fp,fn,tp}')
    
    
site_stats

y_pos[41]: 9.928598095949226%
y_pos[44]: 6.92512398693601%
y_pos[48]: 1.7155643958229736%
y_pos[49]: 5.239958805355304%
y_pos[51]: 0.9277708592777085%


,41,44,48,49,51
recall,0.9600,0.8830,0.3442,0.9222,0.2148
prec,0.1112,0.1681,0.1369,0.1591,0.1208
MCC,0.0976,0.2944,0.1958,0.3162,0.1506
PR_AUC,0.1963,0.3311,0.1588,0.3027,0.0841
roc_auc,0.6724,0.8605,0.8883,0.8964,0.8576
NTN,"1,389.0000","6,282.0000","9,456.0000","6,921.0000","9,762.0000"
NFP,"7,618.0000","3,026.0000",372.0000,"2,555.0000",145.0000
NFN,40.0000,81.0000,113.0000,41.0000,73.0000
NTP,953.0000,611.0000,59.0000,483.0000,20.0000
